In [1]:
import openpnm as op
import openpnm.models.geometry as gm
import openpnm.models.misc as mm
import openpnm.models.physics as pm
import scipy as sp
print(op.__version__)
%matplotlib inline

C:\Users\Tom\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.0.0-b


## Generate Two Networks with Different Spacing

In [2]:
spacing_lg = 0.00006
layer_lg = op.network.Cubic(shape=[10, 10, 1], spacing=spacing_lg)

In [3]:
spacing_sm = 0.00002
layer_sm = op.network.Cubic(shape=[30, 5, 1], spacing=spacing_sm)

## Position Networks Appropriately, then Stitch Together

In [4]:
# Start by assigning labels to each network for identification later
layer_sm['pore.small'] = True
layer_sm['throat.small'] = True
layer_lg['pore.large'] = True
layer_lg['throat.large'] = True
# Next manually offset CL one full thickness relative to the GDL
layer_sm['pore.coords'] -= [0, spacing_sm*5, 0]
layer_sm['pore.coords'] += [0, 0, spacing_lg/2 - spacing_sm/2]  # And shift up by 1/2 a lattice spacing
# Finally, send both networks to stitch which will stitch CL onto GDL
from openpnm.topotools import stitch
stitch(network=layer_lg, donor=layer_sm,
       P_network=layer_lg.pores('bottom'),
       P_donor=layer_sm.pores('top'),
       len_max=0.00005)
combo_net = layer_lg
combo_net.name = 'combo'

## Create Geometry Objects for Each Layer

In [5]:
Ps = combo_net.pores('small')
Ts = combo_net.throats('small')
geom_sm = op.geometry.GenericGeometry(network=combo_net, pores=Ps, throats=Ts)
Ps = combo_net.pores('large')
Ts = combo_net.throats('small', mode='not')
geom_lg = op.geometry.GenericGeometry(network=combo_net, pores=Ps, throats=Ts)

### Add Geometrical Properties to the *Small* Domain
The *small* domain will be treated as a continua, so instead of assigning pore sizes we want the 'pore' to be same size as the lattice cell. 

In [6]:
geom_sm['pore.diameter'] = spacing_sm
geom_sm['pore.area'] = spacing_sm**2
geom_sm['throat.diameter'] = spacing_sm
geom_sm['throat.area'] = spacing_sm**2
geom_sm['throat.length'] = 1e-12  # A very small number to represent nearly 0-length

### Add Geometrical Properties to the *Large* Domain

In [7]:
geom_lg['pore.diameter'] = spacing_lg*sp.rand(combo_net.num_pores('large'))
geom_lg.add_model(propname='pore.area',
                  model=gm.pore_area.sphere)
geom_lg.add_model(propname='throat.diameter',
                  model=mm.misc.from_neighbor_pores,
                  pore_prop='pore.diameter', mode='min')
geom_lg.add_model(propname='throat.area',
                  model=gm.throat_area.cylinder)
geom_lg.add_model(propname='throat.length',
                  model=gm.throat_length.straight)

## Create Phase and Physics Objects

In [8]:
air = op.phases.Air(network=combo_net, name='air')
phys_lg = op.physics.GenericPhysics(network=combo_net, geometry=geom_lg, phase=air)
phys_sm = op.physics.GenericPhysics(network=combo_net, geometry=geom_sm, phase=air)

Add pore-scale models for diffusion to each Physics:

In [9]:
phys_lg.add_model(propname='throat.diffusive_conductance',
                  model=pm.diffusive_conductance.ordinary_diffusion)
phys_sm.add_model(propname='throat.diffusive_conductance',
                  model=pm.diffusive_conductance.ordinary_diffusion)

For the *small* layer we've used a normal diffusive conductance model, which when combined with the diffusion coefficient of air will be equivalent to open-air diffusion.  If we want the *small* layer to have some tortuosity we must account for this:

In [10]:
porosity = 0.5
tortuosity = 2
phys_sm['throat.diffusive_conductance'] *= (porosity/tortuosity)

Note that this extra line is NOT a pore-scale model, so it will be over-written when the `phys_sm` object is regenerated.

### Add a Reaction Term to the Small Layer
A standard n-th order chemical reaction is $ r=k \cdot x^b $, or more generally: $ r = A_1 \cdot x^{A_2} + A_3 $.  This model is available in `OpenPNM.Physics.models.generic_source_terms`, and we must specify values for each of the constants.

In [14]:
# Set Source Term
air['pore.A1'] = -1e-10  # Reaction pre-factor
air['pore.A2'] = 2  # Reaction order
air['pore.A3'] = 0  # A generic offset that is not needed so set to 0
phys_sm.add_model(propname='pore.reaction',
                  model=pm.generic_source_term.power_law,
                  A1='pore.A1', A2='pore.A2', A3='pore.A3',
                  X='pore.mole_fraction', 
                  regen_mode='deferred')

## Perform a Diffusion Calculation

In [15]:
Deff = op.algorithms.ReactiveTransport(network=combo_net, phase=air)
Ps = combo_net.pores(['large', 'right'], mode='intersection')
Deff.set_value_BC(pores=Ps, values=1)
Ps = combo_net.pores('small')
Deff.set_source(propname='pore.reaction', pores=Ps)
Deff.settings['conductance'] = 'throat.diffusive_conductance'
Deff.settings['quantity'] = 'pore.mole_fraction'
Deff.run()

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Running ReactiveTransport
Tolerance not met: 250.00000000000648
Tolerance not met: 146.80800923805344
Tolerance not met: 51.76405941615627
Tolerance not met: 48.529529051275496
Tolerance not met: 81.22663045300838
Tolerance not met: 111.83322214385078
Tolerance not met: 142.55471068621844
Tolerance not met: 155.3832565533095
Tolerance not met: 202.8680980372297
Tolerance not met: 4464.638249174376
Tolerance not met: 5953.987666695672
Tolerance not met: 3878.9594883603654
Tolerance not met: 9095.006815524283
Tolerance not met: 24791.445003412267
Tolerance not met: 16185.43550496443
Tolerance not met: 6229.70093042059
Tolerance not met: 205779.43546763144
Tolerance not met: 158490.7114745998
Tolerance not met: 394767.8016971093
Tolerance not met: 277754.01866163383
Tolerance not met: 76565.52454224268
Tolerance not met: 33770.33514211579
Tolerance not met: 26203.528974746267
Tolerance not met: 15410.74479714

Tolerance not met: 10268.08513332637
Tolerance not met: 40913.75682429537
Tolerance not met: 30137.69375629973
Tolerance not met: 9074.943981681552
Tolerance not met: 9866110.100947326
Tolerance not met: 7399863.612832056
Tolerance not met: 1854213.9042227052
Tolerance not met: 467117.19440413185
Tolerance not met: 137987.33002693125
Tolerance not met: 60329.1214770386
Tolerance not met: 29717.877035998725
Tolerance not met: 11907.506428949528
Tolerance not met: 5829.189062011312
Tolerance not met: 56599.747092704136
Tolerance not met: 42434.191133898836
Tolerance not met: 33128.56019435528
Tolerance not met: 153719.24123615527
Tolerance not met: 137158.2296266897
Tolerance not met: 58945.90550428127
Tolerance not met: 21653.74598959711
Tolerance not met: 44094.71843591385
Tolerance not met: 89905.3807239682
Tolerance not met: 64250.557680156766
Tolerance not met: 93255.31974432597
Tolerance not met: 61319.79783761782
Tolerance not met: 21098.336580139614
Tolerance not met: 23498.69149

Tolerance not met: 22128.718468276875
Tolerance not met: 20409.944292370215
Tolerance not met: 12887.997585475028
Tolerance not met: 104357.32979800788
Tolerance not met: 124233.82579049823
Tolerance not met: 23452486.864472523
Tolerance not met: 17625867.040058322
Tolerance not met: 4452562.004002446
Tolerance not met: 1123302.4248106705
Tolerance not met: 299398.21859964565
Tolerance not met: 95895.38332848581
Tolerance not met: 50141.44570327955
Tolerance not met: 49856.582122664906
Tolerance not met: 1409228.1233688795
Tolerance not met: 1061520.094161365
Tolerance not met: 277704.84188739216
Tolerance not met: 73595.83633699926
Tolerance not met: 62914.84476998568
Tolerance not met: 115970.9856233283
Tolerance not met: 80834.63789985313
Tolerance not met: 28482.945812687547
Tolerance not met: 84822.77693443155
Tolerance not met: 67085.63136626336
Tolerance not met: 47272.391767418085
Tolerance not met: 27964.790834249943
Tolerance not met: 2512393.8893420845
Tolerance not met: 188

Tolerance not met: 26400.02978844661
Tolerance not met: 30314.13409874849
Tolerance not met: 57786.5196076108
Tolerance not met: 38766.7906645156
Tolerance not met: 49463.82859780452
Tolerance not met: 33108.89184641691
Tolerance not met: 28721.18980315922
Tolerance not met: 20892.84849290323
Tolerance not met: 8035.384909862515
Tolerance not met: 3260.7103475180875
Tolerance not met: 6256.970958085992
Tolerance not met: 7240.314915048662
Tolerance not met: 75916.40439939869
Tolerance not met: 282541.73664689437
Tolerance not met: 184797.43320625005
Tolerance not met: 58486.27418511512
Tolerance not met: 58063.15648703356
Tolerance not met: 46826.21173664523
Tolerance not met: 43013.508880521855
Tolerance not met: 36308.84508315689
Tolerance not met: 17758.045638728214
Tolerance not met: 60636.85182205093
Tolerance not met: 91615.75133648499
Tolerance not met: 1287421.9879755455
Tolerance not met: 952583.6361933658
Tolerance not met: 242629.4832726621
Tolerance not met: 78619.974157629

Tolerance not met: 95569.7183828502
Tolerance not met: 150222.07564702575
Tolerance not met: 163474.3742423953
Tolerance not met: 85651.22779899885
Tolerance not met: 46503.0018008027
Tolerance not met: 196544.97533179156
Tolerance not met: 153114.9142999699
Tolerance not met: 78180.73680306644
Tolerance not met: 38881.57159576577
Tolerance not met: 1093338.918750808
Tolerance not met: 852566.0500310336
Tolerance not met: 233330.1409182292
Tolerance not met: 63386.925633818806
Tolerance not met: 18657.978848701252
Tolerance not met: 10109.06485654148
Tolerance not met: 51350.13516091033
Tolerance not met: 713516.4720765967
Tolerance not met: 533302.1771370517
Tolerance not met: 144194.74916167062
Tolerance not met: 41916.04993437824
Tolerance not met: 14943.591128333022
Tolerance not met: 6041.850647684537
Tolerance not met: 5972.521056223326
Tolerance not met: 4858.829857089007
Tolerance not met: 4208.20181844074
Tolerance not met: 4442.045622740468
Tolerance not met: 18093.9049877767

Tolerance not met: 772478.1060019298
Tolerance not met: 447868.2853317313
Tolerance not met: 116109.96294268678
Tolerance not met: 32335.00064221122
Tolerance not met: 142819.85520463777
Tolerance not met: 103490.58637452919
Tolerance not met: 42216.50768644592
Tolerance not met: 38664.93024380021
Tolerance not met: 22354.77284490351
Tolerance not met: 11267.498769039863
Tolerance not met: 891433.2926074484
Tolerance not met: 669615.2394840932
Tolerance not met: 182310.57090099284
Tolerance not met: 60192.627393268966
Tolerance not met: 20978.856759130576
Tolerance not met: 8145.531193565825
Tolerance not met: 78539.41251138884
Tolerance not met: 144591617.219329
Tolerance not met: 108416701.41812232
Tolerance not met: 27107761.11016611
Tolerance not met: 6781893.606777604
Tolerance not met: 1698622.32070018
Tolerance not met: 711686.0460884518
Tolerance not met: 331907.40670518763
Tolerance not met: 615016.2205369245
Tolerance not met: 421368.6652749804
Tolerance not met: 106701.48277

Tolerance not met: 4763.275898106553
Tolerance not met: 68366.8698148155
Tolerance not met: 52996.39692130011
Tolerance not met: 16603.581886211276
Tolerance not met: 5892.223691115525
Tolerance not met: 2285897.8443521336
Tolerance not met: 1715855.190215394
Tolerance not met: 447921.4896183245
Tolerance not met: 126049.10157480737
Tolerance not met: 38251.08795785262
Tolerance not met: 92639.2275926055
Tolerance not met: 73308.4670102564
Tolerance not met: 28459.34385416966
Tolerance not met: 11322.949656497047
Tolerance not met: 26148.50468564644
Tolerance not met: 324094.908810115
Tolerance not met: 255382.26612413433
Tolerance not met: 97124.73199242035
Tolerance not met: 37882.217623065946
Tolerance not met: 21602.0034103021
Tolerance not met: 1778762.8076417933
Tolerance not met: 1332373.520946654
Tolerance not met: 365162.15487315564
Tolerance not met: 115272.189823863
Tolerance not met: 46661.84593242362
Tolerance not met: 129069.35350343182
Tolerance not met: 92478.9023850409

Tolerance not met: 1917296.4191686413
Tolerance not met: 1422563.3482506003
Tolerance not met: 370753.79530272173
Tolerance not met: 103668.01829047043
Tolerance not met: 29737.327099530994
Tolerance not met: 14710.580666969101
Tolerance not met: 7475.431794580823
Tolerance not met: 14006.440965467988
Tolerance not met: 11444.592225282799
Tolerance not met: 28045.58750986916
Tolerance not met: 23408.497478457055
Tolerance not met: 10051.255664592294
Tolerance not met: 32229.5762569056
Tolerance not met: 25365.062631514327
Tolerance not met: 89129.82004054329
Tolerance not met: 67519.46281349604
Tolerance not met: 22447.454099479964
Tolerance not met: 10703.324167550847
Tolerance not met: 5525.055114648437
Tolerance not met: 2715.1643139191788
Tolerance not met: 16357.32339832985
Tolerance not met: 15417.327902506917
Tolerance not met: 12289.755672378637
Tolerance not met: 16613.5055661295
Tolerance not met: 11776.950641169535
Tolerance not met: 5113.268821495371
Tolerance not met: 7169

Tolerance not met: 4135.496192224449
Tolerance not met: 27687.747165147626
Tolerance not met: 23142.78231118075
Tolerance not met: 9748.187018641407
Tolerance not met: 6626.160099136843
Tolerance not met: 8419.422046862333
Tolerance not met: 9992.213226698888
Tolerance not met: 6654.661615115957
Tolerance not met: 130790.02275311697
Tolerance not met: 109644.35665115522
Tolerance not met: 77926.39555413376
Tolerance not met: 61496.63602751889
Tolerance not met: 212798.52871948658
Tolerance not met: 152306.23141095784
Tolerance not met: 42484.92372614095
Tolerance not met: 19670.396970818067
Tolerance not met: 93569.9189621869
Tolerance not met: 68618.745982183
Tolerance not met: 152813.02001334098
Tolerance not met: 110334.55897094612
Tolerance not met: 403946.2151769954
Tolerance not met: 289631.1205522019
Tolerance not met: 74842.88266812195
Tolerance not met: 48037.04982334834
Tolerance not met: 65998.55902091971
Tolerance not met: 37586.987528160294
Tolerance not met: 64177.9038502

RecursionError: maximum recursion depth exceeded in comparison

## Visualize the Concentration Distribution

Save the results to a VTK file for visualization in Paraview:

In [17]:
Deff.results()
op.io.VTK.save(network=combo_net, phases=[air])

And the result would look something like this:

![](https://i.imgur.com/otsd6iX.png)